# Semantic role labeling with spiking neural network using surrogate gradient learning

With code from Zenke's notebook and https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/ and https://discuss.pytorch.org/t/rnn-to-predict-label-from-sequence-in-pytorch/154682
For normalization: https://stackoverflow.com/questions/18380419/normalization-to-bring-in-the-range-of-0-1

Ideally, the input file and potentially already existing data to work with reside in the same directory on disk as this file.

# Importing

In [ ]:
#importing
import os
from os.path import exists

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import seaborn as sns

import torch
import torch.nn as nn

import pandas as pd

import random

from sklearn.preprocessing import MinMaxScaler
import sklearn.preprocessing as prepro
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, cohen_kappa_score, f1_score
from sklearn.model_selection import train_test_split

import re

import pickle

import collections

import io

import copy

import sys

np.random.seed(455)
dtype = torch.float
device = torch.device("cpu")

In [ ]:
#if code is run on a device where saving to disk is not possible or it is easier to print certain variables. Example: Microsoft Azure
#torch.set_printoptions(threshold=sys.maxsize)
#np.set_printoptions(threshold=sys.maxsize)

# Function definitions

In [ ]:
#SURROGATE GRADIENT FUNCTION TO BE USED IN BACKPROPAGATION THROUGH TIME

class SurrGradSpike(torch.autograd.Function):
    """
    Here we implement our spiking nonlinearity which also implements
    the surrogate gradient. By subclassing torch.autograd.Function,
    we will be able to use all of PyTorch's autograd functionality.
    Here we use the normalized negative part of a fast sigmoid
    as this was done in Zenke & Ganguli (2018).
    """

    scale = 100.0 # controls steepness of surrogate gradient

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we compute a step function of the input Tensor
        and return it. ctx is a context object that we use to stash information which
        we need to later backpropagate our error signals. To achieve this we use the
        ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        out = torch.zeros_like(input)
        out[input > 0] = 1.0
        return out

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor we need to compute the
        surrogate gradient of the loss with respect to the input.
        Here we use the normalized negative part of a fast sigmoid
        as this was done in Zenke & Ganguli (2018).
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad = grad_input/(SurrGradSpike.scale*torch.abs(input)+1.0)**2
        return grad

# overwriting spike function with the "SurrGradSpike" nonlinearity which implements a surrogate gradient function
spike_fn  = SurrGradSpike.apply

In [ ]:
#FUNCTION TO RUN THE NETWORK

def run_snn(inputs, nb_hidden, nb_outputs, w1, w2, w3=None, recurrent_on=False):
    num_sequences_input = inputs.shape[0]

    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    syn = torch.zeros((num_sequences_input, nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((num_sequences_input, nb_hidden), device=device, dtype=dtype)

    temp_recurrent = torch.zeros((num_sequences_input, nb_hidden), device=device, dtype=dtype)

    mem_rec = []
    spk_rec = []

    # Compute hidden layer activity
    for t in range(inputs.shape[1]):
        mthr = mem - 1.0
        out = SurrGradSpike.apply(mthr)
        rst = out.detach() # To not propagate the error through the reset term

        new_syn = 1
        if recurrent_on:
            new_syn = alpha * syn + (delta * temp_recurrent + (1 - delta) * h1[:, t])
        else:
            new_syn = alpha * syn + h1[:, t]

        new_mem = (beta * mem + syn) * (1.0 - rst)

        mem_rec.append(mem)
        spk_rec.append(out)

        mem = new_mem
        syn = new_syn
        if recurrent_on:
            temp_recurrent = (w3 @ out.transpose(0, 1)).transpose(0, 1)

    mem_rec = torch.stack(mem_rec, dim=1)
    spk_rec = torch.stack(spk_rec, dim=1)

    # Readout layer
    h2 = torch.einsum("abc,cd->abd", (spk_rec, w2))
    flt = torch.zeros((num_sequences_input, nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((num_sequences_input, nb_outputs), device=device, dtype=dtype)
    out_rec = []
    for t in range(inputs.shape[1]):
        new_flt = alpha * flt + h2[:, t]
        new_out = beta * out + flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec, dim=1)
    other_recs = [mem_rec, spk_rec]
    return out_rec, other_recs

In [ ]:
def process_data(header, until_this=0, data_string=None, exact_length=0, limit_length=0, keep_labels=0):
    # READING THE DATA

    # CHANGE FILE LOCATION TO WHERE THE INPUT FILE RESIDES ON DISK
    f = open(r'./sentenceInput.txt', 'r')

    lineCounter = 0
    header1 = ''
    text = ''
    data = []
    # from https://www.geeksforgeeks.org/read-a-file-line-by-line-in-python/
    while True:
        line = f.readline()
        if line != '':
            if lineCounter == 0:
                header1 = line
            else:
                # from https://note.nkmk.me/en/python-split-rsplit-splitlines-re/
                data.append(line.rstrip('\n'))
                data.append(f.readline().rstrip('\n').split())
                data.append([re.sub("'", "", x) for x in f.readline().strip('\n[]').split(", ")])
                f.readline()
            lineCounter += 1
        else:
            break

    f.close()

    grouped_data = [data[n:n + 3] for n in range(0, len(data), 3)]

    # LIMIT SENTENCE LENGTH TO SPECIFIED LENGTH AND FILTER OUT LESS COMMON LABELS, TO REDUCE COMPLEXITY

    # if exact_length is 0, append sequences of all lengths below limit.
    # if exact_length is a non-zero number, only append sequences of that length
    exact_length = exact_length
    limit_length = limit_length

    temp = []
    if exact_length > 0:
        limit_length = exact_length
    if limit_length > 0:
        for chunk in grouped_data:
            if len(chunk[1]) < (limit_length + 1):
                if exact_length == limit_length:
                    if len(chunk[1]) > (limit_length - 1):
                        temp.append(chunk)
                else:
                    temp.append(chunk)
    else:
        temp = grouped_data

    keep_labels = keep_labels
    grouped_data_filtered = []
    if keep_labels > 0:
        counterObject = collections.Counter([text for chunk in temp for text in chunk[2]])
        difference = set([text for chunk in temp for text in chunk[2]]) - set(
            [twople[0] for twople in counterObject.most_common(keep_labels)])
        for chunk in temp:
            if not difference.intersection(set(chunk[2])):
                grouped_data_filtered.append(chunk)

        print("Five most common roles:", set([twople[0] for twople in counterObject.most_common(5)]))
        print("To be removed:", difference)
    else:
        grouped_data_filtered = temp

    print("All roles:", set([text for chunk in grouped_data for text in chunk[2]]))
    print("Roles that are still present in sentences after filtering out less common ones:",
          set([text for chunk in grouped_data_filtered for text in chunk[2]]))

    # SPLIT DATA INTO INPUT, OUTPUT, AND IDENTIFIERS. ALSO REMOVE DASHES
    identifiers = [chunk[0] for chunk in grouped_data_filtered]
    input_data_old = [chunk[1] for chunk in grouped_data_filtered]
    input_data = []
    for sentence in input_data_old:
        temp_sentence = []
        for word in sentence:
            if "-" in word:
                temp_sentence.append(word[1:])
            else:
                temp_sentence.append(word)
        input_data.append(temp_sentence)
    output_data = [chunk[2] for chunk in grouped_data_filtered]

    # POSSIBLE INPUTS AND POSSIBLE LABELS, MAPPING WORDS TO NUMBERS FOR ONE-HOT ENCODING

    # Join all the sentences together and extract the unique words from the combined sentences
    words_input_extracted = set([text for sublist in input_data for text in sublist])
    words_input = header['fullwordlist']
    words_output = set([text for sublist in output_data for text in sublist])

    # check if all sequences have the same length, otherwise add padding to output word list
    add_padding = False
    only_this_length = len(output_data[1])
    index = 0
    while not add_padding and index < len(output_data):
        if len(output_data[index]) != only_this_length:
            add_padding = True
        index += 1
    if add_padding:
        words_output = {'PADDING'}.union(words_output)

    words = set(list(words_input) + list(words_output))

    # Creating a dictionary that maps integers to the words
    int2word = dict(enumerate(words))

    # Creating another dictionary that maps words to integers
    word2int = {word: ind for ind, word in int2word.items()}

    # Creating a dictionary that maps integers to the words
    int2word_output = dict(enumerate(words_output))

    # Creating another dictionary that maps words to integers
    word2int_output = {word: ind for ind, word in int2word_output.items()}

    # THE MAXIMUM LENGTH IN TERMS OF WORDS IN A SENTENCE

    maxlen = 0
    maxindex = 0
    for i in range(len(input_data)):
        if len(input_data[i]) > maxlen:
            maxlen = len(input_data[i])
            maxindex = i

    # maximum length for input and output are necessarily the same.

    # THE MAXIMUM LENGTH IN TERMS OF CHARACTERS IN A SENTENCE

    maxlen_char = 0
    maxindex_char = (0, 0)
    for i in range(len(input_data)):
        # for j in range(len(input_data[i])):
        temp = ""
        for j in range(len(input_data[i])):
            if not "-" in input_data[i][j]:
                temp = temp + (input_data[i][j])
            else:
                temp = temp + (input_data[i][j][1:])
        if len(temp) > maxlen_char:
            maxlen_char = len(temp)
            maxindex_char = i

    # maximum length for input and output are necessarily the same.

    full_input = copy.deepcopy(input_data)
    full_output = copy.deepcopy(output_data)

    until_this = until_this
    input_data = full_input[:until_this]
    output_data = full_output[:until_this]

    return grouped_data, grouped_data_filtered, identifiers, input_data_old, full_input, input_data, full_output, output_data, words_input_extracted, words_input, words_output, words, int2word, word2int, int2word_output, word2int_output, maxlen, maxindex, maxlen_char, maxindex_char


In [ ]:
def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)

    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [ ]:
def put_dash(words_input, word_wo_dash):
    #Helper function to make comparison between original input and input stripped of "-" possible
    for word in words_input:
        if word.replace('-', '') == word_wo_dash:
            return word
    return 0

In [ ]:
def make_x_y(input_data, output_data, nb_inputs, nb_hidden, nb_outputs, time_step, nb_steps, freq, words_output,
             words_input, maxlen, maxlen_char, input_data_old, word2int_output, until_this, neurons_per_word=0,
             nb_steps_char=0, spikes_words=None):
    if neurons_per_word > 0:  # if you to a spatio-temporal encoding of the input
        nb_inputs = neurons_per_word * len(words_input)
    else:
        nb_inputs = nb_inputs

    spikes_words = spikes_words

    if spikes_words is None:
        spikes_words = []

        if nb_steps_char > 0:
            # CREATING SPIKE TRAINS FOR WORDS THAT ARE VARIABLE LENGTH DEPENDING ON THE INDIVIDUAL WORD LENGTH
            for i in range(len(words_input)):
                if neurons_per_word > 0:
                    if "-" in words_input[i]:
                        spikes_words.append(np.zeros((nb_inputs, len(words_input[i][1:]) * nb_steps_char)))
                    else:
                        spikes_words.append(np.zeros((nb_inputs, len(words_input[i]) * nb_steps_char)))
                else:
                    if "-" in words_input[i]:
                        spikes_words.append(np.zeros((nb_inputs, len(words_input[i][1:]) * nb_steps_char)))
                    else:
                        spikes_words.append(np.zeros((nb_inputs, len(words_input[i]) * nb_steps_char)))

            for i, word in enumerate(words_input):
                # Generate spike pattern for each individual input word
                temp = 1
                if neurons_per_word > 0:
                    temp2 = 1
                    if "-" in words_input[i]:
                        temp2 = np.random.poisson(freq * time_step, size=(
                            neurons_per_word, len(words_input[i][1:]) * nb_steps_char))
                        temp = np.zeros((nb_inputs, len(words_input[i][1:]) * nb_steps_char))
                    else:
                        temp2 = np.random.poisson(freq * time_step, size=(
                            neurons_per_word, len(words_input[i]) * nb_steps_char))
                        temp = np.zeros((nb_inputs, len(words_input[i]) * nb_steps_char))
                    temp[words_input.index(word) * neurons_per_word: (words_input.index(word) + 1) * neurons_per_word,
                    :] = temp2
                else:
                    if "-" in words_input[i]:
                        temp = np.random.poisson(freq * time_step, size=(
                            nb_inputs, len(words_input[i][1:]) * nb_steps_char))
                    else:
                        temp = np.random.poisson(freq * time_step, size=(
                            nb_inputs, len(words_input[i]) * nb_steps_char))
                temp = np.clip(temp, 0, 1)
                # Assign spike pattern for each word to numpy array
                spikes_words[i][:][:] = temp.tolist()
        else:
            # CREATING SPIKE TRAINS FOR WORDS THAT ARE OF FIXED LENGTH FOR EVERY WORD
            spikes_words = 1
            if neurons_per_word > 0:
                spikes_words = np.random.poisson(freq * time_step, size=(len(words_input), neurons_per_word, nb_steps))
                spikes_words = np.clip(spikes_words, 0, 1)
                spikes_words = torch.tensor(spikes_words.transpose((2, 0, 1)).reshape(nb_steps, -1))
            else:
                spikes_words = np.random.poisson(freq * time_step, size=(len(words_input), nb_inputs, nb_steps))
                spikes_words = np.clip(spikes_words, 0, 1)

    # SENTENCE SPIKE TRAINS

    sentence_spike_patterns = 1

    if nb_steps_char > 0:
        # CREATING SPIKE TRAINS FOR SENTENCES WITH WORDS THAT ARE OF VARIABLE LENGTH
        # ALL SENTENCE SPIKE TRAINS ARE PADDED TO THE SAME LENGTH
        for i, sentence in enumerate(input_data):
            if i % 50 == 49:
                print("processing...", str(i + 1))
                
            sentence_spike_pattern = 1

            for j, word in enumerate(sentence):
                spike_word = np.transpose(spikes_words[words_input.index(put_dash(words_input, word))][:])

                if j == 0:
                    sentence_spike_pattern = spike_word
                else:
                    sentence_spike_pattern = np.concatenate((sentence_spike_pattern, spike_word), axis=0)
                # sentence_spike_pattern.extend(spike_word)
                # sentence_spike_pattern[j * nb_steps : (j + 1) * nb_steps, words_input.index(word) : (words_input.index(word)+1)] = spike_word

            if not sentence_spike_pattern.shape[0] == (nb_steps_char * maxlen_char):
                sentence_spike_pattern = np.concatenate((sentence_spike_pattern, np.zeros(
                    (((nb_steps_char * maxlen_char) - sentence_spike_pattern.shape[0]), nb_inputs))), axis=0)

            sentence_spike_pattern = torch.tensor(sentence_spike_pattern,
                                                  dtype=dtype)

            if i == 0:
                sentence_spike_patterns = torch.reshape(sentence_spike_pattern, (
                    1, sentence_spike_pattern.shape[0], sentence_spike_pattern.shape[1]))
            else:
                sentence_spike_patterns = torch.cat((sentence_spike_patterns, sentence_spike_pattern.unsqueeze(0)),
                                                    dim=0)

        print("sentence_spike_patterns.shape", sentence_spike_patterns.shape)
    else:
        # CREATING SPIKE TRAINS FOR SENTENCES WHERE WORDS ARE OF FIXED LENGTH.
        # ALL SENTENCE SPIKE TRAINS ARE PADDED TO THE SAME LENGTH

        for i, sentence in enumerate(input_data_old[:until_this]):
            if i % 50 == 49:
                print("processing...", str(i + 1))

            sentence_spike_pattern = np.zeros((nb_steps * maxlen, nb_inputs))

            for j, word in enumerate(sentence):
                if neurons_per_word > 0:
                    spike_word = spikes_words[:, words_input.index(word) * neurons_per_word: (words_input.index(
                        word) + 1) * neurons_per_word]
                    sentence_spike_pattern[j * nb_steps: (j + 1) * nb_steps,
                    words_input.index(word) * neurons_per_word: (words_input.index(
                        word) + 1) * neurons_per_word] = spike_word
                else:
                    spike_word = spikes_words[words_input.index(word), :, :]
                    sentence_spike_pattern[j * nb_steps: (j + 1) * nb_steps, :] = np.transpose(spike_word)

            sentence_spike_pattern = torch.tensor(sentence_spike_pattern, dtype=dtype)
            if i == 0:
                sentence_spike_patterns = torch.reshape(sentence_spike_pattern, (
                    1, sentence_spike_pattern.shape[0], sentence_spike_pattern.shape[1]))
            else:
                sentence_spike_patterns = torch.cat((sentence_spike_patterns, sentence_spike_pattern.unsqueeze(0)),
                                                    dim=0)
        print("sentence_spike_patterns.shape", sentence_spike_patterns.shape)

    # CONVERTING OUTPUT LABELS TO NUMBERS CORRESPONDING TO THE RESPECTIVE LABELS

    output_int = []
    for i in range(len(output_data)):
        temp = [word2int_output[word] for word in output_data[i]]
        if 'PADDING' in words_output:  # add_padding:
            output_int.append(temp + [word2int_output['PADDING']] * (maxlen - len(temp)))
        else:
            output_int.append(temp)

    # MAKE THE OUTPUT LABELS INTO ONE-HOT ENCODED DATA

    # TODO: from RNN_walkthrough

    dict_size = len(word2int_output)
    seq_len = maxlen

    output_seq = one_hot_encode(output_int, dict_size, seq_len, len(output_int))
    print("Output shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(output_seq.shape))

    x_data = sentence_spike_patterns
    y_data = torch.tensor(output_seq, dtype=dtype)

    return x_data, y_data, spikes_words

In [ ]:
def make_weights(beta, nb_inputs, nb_hidden, nb_outputs):
    weight_scale = 7 * (1.0 - beta)

    w1 = torch.empty((nb_inputs, nb_hidden), device=device, dtype=dtype, requires_grad=True)
    torch.nn.init.normal_(w1, mean=0.0, std=weight_scale / np.sqrt(nb_inputs))

    w2 = torch.empty((nb_hidden, nb_outputs), device=device, dtype=dtype, requires_grad=True)
    torch.nn.init.normal_(w2, mean=0.0, std=weight_scale / np.sqrt(nb_hidden))

    w3 = torch.empty((nb_hidden, nb_hidden), device=device, dtype=dtype, requires_grad=True)
    torch.nn.init.normal_(w3, mean=0.0, std=weight_scale / np.sqrt(nb_hidden))
    return w1, w2, w3

In [ ]:
def optimization(w1, w2, w3, input_data, x_data, y_data, epochs, learning_rate=2e-3, recurrent_on=False):
    params = [w1, w2, w3]  # The parameters we want to optimize
    optimizer = torch.optim.Adam(params, lr=2e-3, betas=(0.9, 0.999))  # The optimizer we are going to use

    log_softmax_fn = nn.LogSoftmax(dim=1)  # The log softmax function across output units
    loss_fn = nn.CrossEntropyLoss()  # The cross entropy loss function

    # The optimization loop
    loss_hist = []
    for e in range(epochs):
        loss_over_epoch = []
        print("new epoch number", e)
        for i, sequence in enumerate(x_data):
            optimizer.zero_grad()
            sequence = torch.unsqueeze(sequence, dim=0)
            output, _ = run_snn(sequence, nb_hidden, nb_outputs, w1, w2, w3, recurrent_on)
            chunks = []
            for word in input_data[i]:
                chunks.append(len(word) * nb_steps_char)
            if sum(chunks) < x_data.shape[1]:
                chunks.append((x_data.shape[1] - sum(chunks)))
            seq_split = torch.split(torch.squeeze(output), split_size_or_sections=chunks, dim=0)

            for j, word in enumerate(seq_split):
                if j < y_data.shape[1]:
                    log_p_y = log_softmax_fn(word[(word.shape[0] - 1), :][None, :]).squeeze()
                    y = y_data[i][j]
                    loss_val = loss_fn(log_p_y, y)
                    loss_val.backward(retain_graph=True)
                    loss_over_epoch.append(loss_val.item())

            optimizer.step()
        loss_hist.append(np.mean(loss_over_epoch))
        print("Epoch:", str(e) + ",", "Loss:", loss_hist[-1])

    return w1, w2, w3, loss_hist

In [ ]:
def voltage_pred(output, input_data, maxlen, words_output, x_data, y_data, word2int_output):
    # turning the voltage traces into output labels
    if type(output) is tuple:
        output, _ = output
        output = torch.squeeze(output)

    predictions = torch.zeros(len(output), maxlen, len(words_output), dtype=torch.int64)
    for sequence_index, sequence_data in enumerate(output):
        chunks = []
        for word in input_data[sequence_index]:
            chunks.append(len(word) * nb_steps_char)
        if sum(chunks) < x_data.shape[1]:
            chunks.append((x_data.shape[1] - sum(chunks)))
        sequence_data_splitted = torch.split(sequence_data, split_size_or_sections=chunks, dim=0)
        for word_index, word in enumerate(sequence_data_splitted):
            if word_index < y_data.shape[1]:
                predictions[sequence_index, word_index] = word[(word.shape[0] - 1), :].argmax()
    dict_size = len(word2int_output)
    seq_len = maxlen
    predictions = one_hot_encode(predictions, dict_size, seq_len, len(output))
    return predictions

In [ ]:
def accuracy_f1(predictions, y_data):
    # calculating the accuracy and f1 score using the output predictions and the actual labels
    correct = 0
    total = 0
    for i in range(y_data.shape[0]):
        act_label = np.argmax(y_data[i])
        pred_label = np.argmax(predictions[i])
        if (act_label == pred_label):
            correct += 1
        total += 1
    accuracy = (correct / total)
    f1 = f1_score(np.reshape(predictions, (predictions.shape[0] * predictions.shape[1], predictions.shape[2])),
                  np.reshape(y_data, (y_data.shape[0] * y_data.shape[1], y_data.shape[2])), average='weighted')
    return accuracy, f1

In [ ]:
def plot_voltage_traces(mem, spk=None, dim=(3,5), spike_height=5):
    gs=GridSpec(*dim)
    if spk is not None:
        dat = 1.0*mem
        dat[spk>0.0] = spike_height
        dat = dat.detach().cpu().numpy()
    else:
        dat = mem.detach().cpu().numpy()
    for i in range(np.prod(dim)):
        if i==0: a0=ax=plt.subplot(gs[i])
        else: ax=plt.subplot(gs[i],sharey=a0)
        ax.plot(dat[i])
        ax.axis("off")

In [ ]:
def split_train_test(x_complete, y_complete, input_complete, output_complete, train_indices, test_indices):
    x_data = x_complete[train_indices]
    y_data = y_complete[train_indices]
    temp = []
    for index in train_indices:
        temp.append(input_complete[index])
    input_data = temp
    temp = []
    for index in train_indices:
        temp.append(output_complete[index])
    output_data = temp

    x_test = x_complete[test_indices]
    y_test = y_complete[test_indices]
    temp = []
    for index in test_indices:
        temp.append(input_complete[index])
    input_test = temp
    temp = []
    for index in test_indices:
        temp.append(output_complete[index])
    output_test = temp

    return x_data, y_data, input_data, output_data, x_test, y_test, input_test, output_test

# Training the models

In [ ]:
header = {'roles': 18, 'categorylist': ['DET', 'NOUNA', 'NOUNI', 'VERBUNACC', 'VERBLOC', 'VERBUNERG', 'VERBAPT', 'VERBTET', 'VERBCM', 'VERBD', 'ADJ', 'AUX', 'PRO', 'TO', 'ON', 'PER', 'INF', 'BY', 'PAR', 'BEING'], 'messages': 14, 'messagelist': ['animate intransitive', 'inanimate intransitive', 'agent-patient transitive active', 'agent-patient transitive passive', 'theme-experiencer transitive active', 'theme-experiencer transitive passive', 'transfer dative active, prepositional', 'transfer dative passive, prepositional', 'transfer dative active, double object', 'transfer dative passive, double object', 'caused motion active', 'caused motion passive', 'locative active', 'PLOC'], 'fullwordlist': ['the', 'a', 'prn', 'man', 'woman', 'cat', 'dog', 'boy', 'girl', 'father', 'mother', 'ball', 'stick', 'apple', 'cake', 'orange', 'banana', 'phone', 'cup', 'sleep', 'jump', 'dance', 'sit', 'go', 'walk', 'drive', 'run', 'open', 'close', 'break', 'smash', 'kick', 'chase', 'touch', 'eat', 'scare', 'surprise', 'bother', 'hurt', 'put', 'hit', 'carry', 'push', 'give', 'throw', 'show', 'present', 'red', 'beautiful', 'nice', 'big', 'small', 'great', 'old', 'smelly', 'is', 'was', 'are', 'were', 'he', 'she', 'it', 'him', 'her', 'they', 'them', 'to', 'on', '.', '-ing', '-ss', '-ed', '-s', 'by', '-par', 'being'], 'words': 76, 'allroles': set(['AGENT_N0', 'AGENT_N1', 'AGENT_N2', 'EOS', 'ACTION_V0', 'RECIPIENT_N2', 'RECIPIENT_N0', 'RECIPIENT_N1', 'GOAL_N1', 'GOAL_N0', 'GOAL_N2', 'EXPERIENCER_N1', 'EXPERIENCER_N0', 'THEME_N0', 'THEME_N1', 'THEME_N2', 'PATIENT_N0', 'PATIENT_N1']), 'categories': 20}

In [ ]:
recurrent_experiment = [False, True]

In [ ]:
# until_this represents how many sequences will be generated at most in total.
# IT NEEDS TO BE AT LEAST A TWO-DIGIT, NUMBER.
until_this_here = 10
epochs_all = 1
train_perc = 0.71

In [ ]:
# declaring variables so they can be used later
neurons_per_word,nb_inputs,nb_hidden,nb_outputs,time_step,nb_steps,nb_steps_char,freq,x_data,y_data,input_data,output_data,maxlen,words_output,words_input,word2int_output,x_test,y_test,input_test,output_test = 1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1

In [ ]:
loss_experiment = []
weights_experiment = []
acc_experiment = []
acc_test_experiment = []

## Temporal models (one feedforward, one recurrent)

In [ ]:
if exists("./data_temporal.pkl"):
    with open('./data_temporal.pkl', 'rb') as f:
        nb_inputs, nb_hidden, nb_outputs, time_step, nb_steps, freq, words_input_extracted, words_input, words_output, word2int, int2word, int2word_output, word2int_output, maxlen, maxlen_char, complete_y, input_data_old, complete_input, complete_output, train_indices, test_indices, until_this_here, neurons_per_word, nb_steps_char, spikes_words = pickle.load(
            f)
    print("read done data_temporal")
    x_data, y_data, spikes_words = make_x_y(complete_input, complete_output, nb_inputs, nb_hidden, nb_outputs,
                                            time_step, nb_steps, freq, words_output, words_input, maxlen,
                                            maxlen_char,
                                            input_data_old, word2int_output, until_this_here,
                                            nb_steps_char=nb_steps_char, spikes_words=spikes_words)
    complete_x = copy.deepcopy(x_data)
    complete_y = copy.deepcopy(y_data)
    x_data, y_data, input_data, output_data, x_test, y_test, input_test, output_test = split_train_test(complete_x,
                                                                                                        complete_y,
                                                                                                        complete_input,
                                                                                                        complete_output,
                                                                                                        train_indices,
                                                                                                        test_indices)
    print("num inputs:", str(nb_inputs) + ",", "num hidden:", str(nb_hidden) + ",", "num outputs:", nb_outputs)
    print("After train test split x_data.shape", x_data.shape)
    print("After train test split y_data.shape", y_data.shape)
    print("After train test split len(input_data)", len(input_data))
    print("After train test split len(output_data)", len(output_data))

else:
    grouped_data, grouped_data_filtered, identifiers, input_data_old, full_input, input_data, full_output, output_data, words_input_extracted, words_input, words_output, words, int2word, word2int, int2word_output, word2int_output, maxlen, maxindex, maxlen_char, maxindex_char = process_data(
        header, until_this_here, limit_length=10, keep_labels=5)
    complete_input = copy.deepcopy(input_data)
    complete_output = copy.deepcopy(output_data)
    # print("words_input", words_input)
    # print("words_input_extracted", words_input_extracted)
    # print("len(words_input)", len(words_input))
    # print("len(words_input_extracted)", len(words_input_extracted))

    neurons_per_word = 0  # neurons_experiment[i]
    nb_inputs = 100  # neurons_per_word * len(words_input)
    nb_hidden = 25
    nb_outputs = len(words_output)
    print("num inputs:", str(nb_inputs) + ",", "num hidden:", str(nb_hidden) + ",", "num outputs:", nb_outputs)
    time_step = 1e-3
    nb_steps = 200
    nb_steps_char = 50
    # batch_size = until_this_here
    freq = 100  # Hz

    x_data, y_data, spikes_words = make_x_y(complete_input, complete_output, nb_inputs, nb_hidden, nb_outputs,
                                            time_step, nb_steps, freq, words_output, words_input, maxlen,
                                            maxlen_char,
                                            input_data_old, word2int_output, until_this_here,
                                            nb_steps_char=nb_steps_char)

    train_indices = random.sample(range(0, until_this_here), int(until_this_here * train_perc))
    test_indices = [num for num in range(0, until_this_here)]
    test_indices = [x for x in test_indices if x not in train_indices]

    complete_x = copy.deepcopy(x_data)
    complete_y = copy.deepcopy(y_data)

    x_data, y_data, input_data, output_data, x_test, y_test, input_test, output_test = split_train_test(complete_x,
                                                                                                        complete_y,
                                                                                                        complete_input,
                                                                                                        complete_output,
                                                                                                        train_indices,
                                                                                                        test_indices)

    print("After train test split x_data.shape", x_data.shape)
    print("After train test split y_data.shape", y_data.shape)
    print("After train test split len(input_data)", len(input_data))
    print("After train test split len(output_data)", len(output_data))

    with open('./data_temporal.pkl', 'wb') as f:
        pickle.dump(
            (nb_inputs, nb_hidden, nb_outputs, time_step, nb_steps, freq, words_input_extracted,
             words_input, words_output, word2int, int2word, int2word_output, word2int_output, maxlen,
             maxlen_char, complete_y, input_data_old, complete_input, complete_output, train_indices,
             test_indices, until_this_here, neurons_per_word, nb_steps_char, spikes_words), f)
    print("write done data_temporal")

In [ ]:
print(words_input)
print(words_input_extracted)
print(len(words_input))
print(len(words_input_extracted))

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))
delta = 0.2

### Feedforward temporal model

In [ ]:
print("\nfeedforward temporal model")
w1_temp_1, w2_temp_1, w3_temp_1 = make_weights(beta, nb_inputs, nb_hidden, nb_outputs)

w1_temp_1, w2_temp_1, w3_temp_1, loss_hist_temp_1 = optimization(w1_temp_1, w2_temp_1, w3_temp_1, input_data,
                                                                 x_data, y_data, epochs=epochs_all,
                                                                 learning_rate=5e-4,
                                                                 recurrent_on=False)

accuracy_temp_1, f1_temp_1 = accuracy_f1(
    voltage_pred(run_snn(x_data, nb_hidden, nb_outputs, w1_temp_1, w2_temp_1, w3_temp_1, recurrent_on=False),
                 input_data, maxlen, words_output, x_data, y_data, word2int_output), y_data)
accuracy_test_temp_1, f1_test_temp_1 = accuracy_f1(
    voltage_pred(run_snn(x_test, nb_hidden, nb_outputs, w1_temp_1, w2_temp_1, w3_temp_1, recurrent_on=False),
                 input_test, maxlen, words_output, x_test, y_test, word2int_output), y_test)

loss_experiment.append(loss_hist_temp_1)
weights_experiment.append([w1_temp_1, w2_temp_1, w3_temp_1])
acc_experiment.append([accuracy_temp_1, f1_temp_1])
acc_test_experiment.append([accuracy_test_temp_1, f1_test_temp_1])

with open('./results_after_temporal1.pkl', 'wb') as f:
    pickle.dump((loss_experiment, weights_experiment, acc_experiment, acc_test_experiment), f)

print("write done results_after_temporal1")

### Recurrent temporal model

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))
delta = 0.2

#recurrent temporal model

print("\nrecurrent temporal model")
w1_temp_2, w2_temp_2, w3_temp_2 = make_weights(beta, nb_inputs, nb_hidden, nb_outputs)

w1_temp_2, w2_temp_2, w3_temp_2, loss_hist_temp_2 = optimization(w1_temp_2, w2_temp_2, w3_temp_2, input_data,
                                                                 x_data, y_data, epochs=epochs_all,
                                                                 learning_rate=5e-4,
                                                                 recurrent_on=True)

accuracy_temp_2, f1_temp_2 = accuracy_f1(
    voltage_pred(run_snn(x_data, nb_hidden, nb_outputs, w1_temp_2, w2_temp_2, w3_temp_2, recurrent_on=True),
                 input_data, maxlen, words_output, x_data, y_data, word2int_output), y_data)
accuracy_test_temp_2, f1_test_temp_2 = accuracy_f1(
    voltage_pred(run_snn(x_test, nb_hidden, nb_outputs, w1_temp_2, w2_temp_2, w3_temp_2, recurrent_on=True),
                 input_test, maxlen, words_output, x_test, y_test, word2int_output), y_test)

loss_experiment.append(loss_hist_temp_2)
weights_experiment.append([w1_temp_2, w2_temp_2, w3_temp_2])
acc_experiment.append([accuracy_temp_2, f1_temp_2])
acc_test_experiment.append([accuracy_test_temp_2, f1_test_temp_2])

with open('./results_after_temporal2.pkl', 'wb') as f:
    pickle.dump((loss_experiment, weights_experiment, acc_experiment, acc_test_experiment), f)

print("write done results_after_temporal2")

## Spatio-temporal models (one feedforward, one recurrent)

In [ ]:
if exists("./data_spatio_temporal.pkl"):
    with open('./data_spatio_temporal.pkl', 'rb') as f:
        nb_inputs, nb_hidden, nb_outputs, time_step, nb_steps, freq, words_input_extracted, words_input, words_output, word2int, int2word, int2word_output, word2int_output, maxlen, maxlen_char, complete_y, input_data_old, complete_input, complete_output, train_indices, test_indices, until_this_here, neurons_per_word, nb_steps_char, spikes_words = pickle.load(f)
    print("read done data_spatio_temporal")
    x_data, y_data, spikes_words = make_x_y(complete_input, complete_output, nb_inputs, nb_hidden, nb_outputs,
                                            time_step, nb_steps, freq, words_output, words_input, maxlen,
                                            maxlen_char,
                                            input_data_old, word2int_output, until_this_here,
                                            nb_steps_char=nb_steps_char, spikes_words=spikes_words)
    complete_x = copy.deepcopy(x_data)
    complete_y = copy.deepcopy(y_data)
    x_data, y_data, input_data, output_data, x_test, y_test, input_test, output_test = split_train_test(complete_x,
                                                                                                        complete_y,
                                                                                                        complete_input,
                                                                                                        complete_output,
                                                                                                        train_indices,
                                                                                                        test_indices)
    print("num inputs:", str(nb_inputs) + ",", "num hidden:", str(nb_hidden) + ",", "num outputs:", nb_outputs)
    print("After train test split x_data.shape", x_data.shape)
    print("After train test split y_data.shape", y_data.shape)
    print("After train test split len(input_data)", len(input_data))
    print("After train test split len(output_data)", len(output_data))

else:
    neurons_per_word = 5  # neurons_experiment[i]
    nb_inputs = neurons_per_word * len(words_input)
    nb_hidden = 25
    nb_outputs = len(words_output)
    print("num inputs:", str(nb_inputs) + ",", "num hidden:", str(nb_hidden) + ",", "num outputs:", nb_outputs)
    time_step = 1e-3
    nb_steps = 200
    nb_steps_char = 50
    # batch_size = until_this_here
    freq = 100  # Hz

    x_data = 1
    y_data = 1
    print(x_data, y_data)

    x_data, y_data, spikes_words = make_x_y(complete_input, complete_output, nb_inputs, nb_hidden, nb_outputs,
                                            time_step, nb_steps, freq, words_output, words_input, maxlen,
                                            maxlen_char,
                                            input_data_old, word2int_output, until_this_here,
                                            neurons_per_word=neurons_per_word, nb_steps_char=nb_steps_char)

    train_indices = random.sample(range(0, until_this_here), int(until_this_here * train_perc))
    test_indices = [num for num in range(0, until_this_here)]
    test_indices = [x for x in test_indices if x not in train_indices]

    complete_x = copy.deepcopy(x_data)
    complete_y = copy.deepcopy(y_data)

    x_data, y_data, input_data, output_data, x_test, y_test, input_test, output_test = split_train_test(complete_x,
                                                                                                        complete_y,
                                                                                                        complete_input,
                                                                                                        complete_output,
                                                                                                        train_indices,
                                                                                                        test_indices)

    print("After train test split x_data.shape", x_data.shape)
    print("After train test split y_data.shape", y_data.shape)
    print("After train test split len(input_data)", len(input_data))
    print("After train test split len(output_data)", len(output_data))

    with open('./data_spatio_temporal.pkl', 'wb') as f:
        pickle.dump((nb_inputs, nb_hidden, nb_outputs, time_step, nb_steps, freq, words_input_extracted,
                     words_input, words_output, word2int, int2word, int2word_output, word2int_output, maxlen,
                     maxlen_char, complete_y, input_data_old, complete_input, complete_output, train_indices,
                     test_indices, until_this_here, neurons_per_word, nb_steps_char, spikes_words), f)

    print("write done data_spatio_temporal")

### Feedforward spatio-temporal model

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha = float(np.exp(-time_step / tau_syn))
beta = float(np.exp(-time_step / tau_mem))
delta = 0.2

In [ ]:
print("\nfeedforward spatio-temporal model")

w1_spatemp_1, w2_spatemp_1, w3_spatemp_1 = make_weights(beta, nb_inputs, nb_hidden, nb_outputs)

w1_spatemp_1, w2_spatemp_1, w3_spatemp_1, loss_hist_spatemp_1 = optimization(w1_spatemp_1, w2_spatemp_1,
                                                                             w3_spatemp_1, input_data, x_data,
                                                                             y_data, epochs=epochs_all,
                                                                             learning_rate=5e-4,
                                                                             recurrent_on=False)

accuracy_spatemp_1, f1_spatemp_1 = accuracy_f1(voltage_pred(
    run_snn(x_data, nb_hidden, nb_outputs, w1_spatemp_1, w2_spatemp_1, w3_spatemp_1, recurrent_on=False),
    input_data, maxlen, words_output, x_data, y_data, word2int_output), y_data)
accuracy_test_spatemp_1, f1_test_spatemp_1 = accuracy_f1(voltage_pred(
    run_snn(x_test, nb_hidden, nb_outputs, w1_spatemp_1, w2_spatemp_1, w3_spatemp_1, recurrent_on=False),
    input_test, maxlen, words_output, x_test, y_test, word2int_output), y_test)

loss_experiment.append(loss_hist_spatemp_1)
weights_experiment.append([w1_spatemp_1, w2_spatemp_1, w3_spatemp_1])
acc_experiment.append([accuracy_spatemp_1, f1_spatemp_1])
acc_test_experiment.append([accuracy_test_spatemp_1, f1_test_spatemp_1])

with open('./results_after_spatio_temporal1.pkl', 'wb') as f:
    pickle.dump((loss_experiment, weights_experiment, acc_experiment, acc_test_experiment), f)

print("write done results_after_spatio_temporal1")

### Recurrent spatio-temporal model

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha = float(np.exp(-time_step / tau_syn))
beta = float(np.exp(-time_step / tau_mem))
delta = 0.2

In [ ]:
print("\nrecurrent spatio-temporal model")

w1_spatemp_2, w2_spatemp_2, w3_spatemp_2 = make_weights(beta, nb_inputs, nb_hidden, nb_outputs)

w1_spatemp_2, w2_spatemp_2, w3_spatemp_2, loss_hist_spatemp_2 = optimization(w1_spatemp_2, w2_spatemp_2,
                                                                             w3_spatemp_2, input_data, x_data,
                                                                             y_data, epochs=epochs_all,
                                                                             learning_rate=5e-4,
                                                                             recurrent_on=True)

accuracy_spatemp_2, f1_spatemp_2 = accuracy_f1(voltage_pred(
    run_snn(x_data, nb_hidden, nb_outputs, w1_spatemp_2, w2_spatemp_2, w3_spatemp_2, recurrent_on=True), input_data,
    maxlen, words_output, x_data, y_data, word2int_output), y_data)
accuracy_test_spatemp_2, f1_test_spatemp_2 = accuracy_f1(voltage_pred(
    run_snn(x_test, nb_hidden, nb_outputs, w1_spatemp_2, w2_spatemp_2, w3_spatemp_2, recurrent_on=True), input_test,
    maxlen, words_output, x_test, y_test, word2int_output), y_test)

loss_experiment.append(loss_hist_spatemp_2)
weights_experiment.append([w1_spatemp_2, w2_spatemp_2, w3_spatemp_2])
acc_experiment.append([accuracy_spatemp_2, f1_spatemp_2])
acc_test_experiment.append([accuracy_test_spatemp_2, f1_test_spatemp_2])

## Results for all models

In [ ]:
with open('./results_after_spatio_temporal2.pkl', 'wb') as f:
    pickle.dump((loss_experiment, weights_experiment, acc_experiment, acc_test_experiment), f)

print("write done results_after_spatio_temporal2")

print("length loss_experiment:", len(loss_experiment))
print("length weights_experiment:", len(weights_experiment))
print("length acc_experiment:", len(acc_experiment))
print("length acc_test_experiment:", len(acc_test_experiment))

print("Training and test accuracies and F1 scores per model:")
for index, accuracy_and_f1 in enumerate(acc_experiment):
    print("Model number " + str(index + 1) + " with training accuracy", accuracy_and_f1[0], "and training F1 score",
          accuracy_and_f1[1])
for index, accuracy_and_f1 in enumerate(acc_test_experiment):
    print("Model number " + str(index + 1) + " with test accuracy", accuracy_and_f1[0], "and test F1 score",
          accuracy_and_f1[1])